# Pyecharts进阶使用

* 哈喽，欢迎来到第三章，此notebook将会带领大家在Pyecharts中实现一些进阶的可视化效果；

* 常规的配置只会让你的图表不出错，但如果你希望你的可视化作品能够让人眼前一亮，往往还需要一些小技巧在里面；

---

## 选对合适的图表

* 在进行可视化之前，我们必须得想清楚，我们的数据是什么，是要来解决什么问题，需要通过图表来做什么。

* 选对一个合适的图表，比任何美化图表的技巧都管用；

* **参考下方表格**

| 对比关系|	定义	| 示例	| 图表形式 |
|----|-----|-----|----|
| 占比	| 个体在整体中占有的比例 |	A 类业务的收入在整个公司中占据了 50%	| 饼图、环形图、玫瑰图、百分比堆积柱形图、百分比堆积条形图、树状图、百分比瀑布图、矩形树图 |
|比较与排序	| 显示 A 和 B 之间的比较；对不同的对象进行排名 | A 区域在 8 月的销售额远超其他 5 个区域 |	柱形图、条形图、子弹图、雷达图、词云图、哑铃图、棒棒糖图
| 时间趋势 |	业务指标随着时间的推移而变化	| 销售额在 5 年内的增长情况 |	折线图、面积图 |
| 频率分布 |	同一指标的不同维度分析	| 某公司人员的年龄段分布、收入分布	、 | 直方图、箱线图、人口金字塔 |
| 位置分布 | 根据地理位置数据，通过地图展示不同分布特征 |	某公司在全国各省份的销售额、人口密度	| 地图、流向地图、点地图、热力地图 |
| 相关性	| 某A、B和C之间的关系	| 通胀与失业率、收入与平均寿命	| 散点图、气泡图、流程图、甘特图、组织结构图 |




## 双Y轴（折线图+柱状图）

* 在实际工作中，我们可能会遇到需要将**不同量级**的多个系列数据放到同一个图中的需求；

* 比如我们在成本核算的时候需要展示当天的成本是多少，收入是多少，然后对应当天的ROI是多少，成本和收入通常都是比较大的数值，而ROI通常是约等于1的比率，这时最好的办法便是成本和收入通过直方图展示，ROI通过折线图来展示，折线图和直方图分别对应不同的Y轴。


In [1]:
from pyecharts.charts import *
from pyecharts import options as opts
import random

x_data = ['香蕉', '梨子', '水蜜桃', '核桃', '西瓜', '苹果']
y_data_1 = [random.randint(10, 50) for _ in range(len(x_data))]
y_data_2 = [random.randint(100, 500) for _ in range(len(x_data))]

# 新建一个直方图Bar
bar = Bar()
bar.add_xaxis(x_data)
# 添加一个Y轴
bar.extend_axis(yaxis=opts.AxisOpts())
# 通过yaxis_index指定Y轴
bar.add_yaxis(
    '左边Y轴',   # 系列名称
    y_data_1,   # 添加数据
    yaxis_index=0,   # 指定y轴，等于0时可以省略
    color='rgba(255, 0, 0, .5)'  # 未避免柱状图遮挡住折线，我们可以调整透明度
)

# 新建一个折线图Line
line = Line()

line.add_xaxis(x_data)
# 将line数据通过yaxis_index指向后添加的Y轴
line.add_yaxis(
    '右边Y轴',   # 系列名称
    y_data_2,   # 添加折线图的数据
    yaxis_index=1  # 指定使用的Y轴
)

# overlap 将两个（Bar和Line层叠在一起）
bar.overlap(line)
# 渲染图表
bar.render_notebook()


## 堆叠效果

* 这边以柱状图为例，对于直角坐标系图表，使用方法都是一致的；

* 在add_yaxis()中有个参数stack ，控制不同系列数据之间是否堆叠，假如所有系列都stack参数全为None，则都不堆叠，如果stack不为空，那么stack值相同都系列数据将会堆叠在一起；

In [2]:
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.faker import Faker

"""
Faker 是Pyechart中提供的示例数据的class
会根据图表类型随机生成一些样例数据
"""

bar = Bar(
    init_opts=opts.InitOpts(
        theme='light',
        width='1000px',
        height='600px')
        )

# 添加分类（x轴）的数据
bar.add_xaxis(Faker.choose())

# stack值一样的系列会堆叠在一起
# A系列和B系列都是stack1，会产生堆叠
# C系列为stack2 不与AB堆叠
bar.add_yaxis('A', Faker.values(), stack='stack1')
bar.add_yaxis('B', Faker.values(), stack='stack1')
bar.add_yaxis('C', Faker.values(), stack='stack2')

bar.render_notebook()


## 阴影效果

* 给元素添加一点阴影，让图表更加立体

In [3]:
from pyecharts.charts import *
from pyecharts import options as opts
import random


# 线的样式，通过字典设置
# Pyecharts中封装的类支持的参数有限
# 想要实现阴影效果只能通过字典进行传参
line_style = {
    'normal': {
        'width': 4,  # 设置线宽
        'shadowColor': 'rgba(155, 18, 184, .3)',  # 阴影颜色
        'shadowBlur': 10,  # 光影大小
        'shadowOffsetY': 10,  # Y轴方向阴影偏移
        'shadowOffsetX': 10,  # x轴方向阴影偏移
        'curve': 0.5  # 线弯曲程度，1表示不弯曲
    }
}

x_data = ["2021/08/{}".format(i + 1) for i in range(31)]

# 随机生成点数据
y_data_1 = [i + random.randint(10, 20) for i in range(len(x_data))]
y_data_2 = [i + random.randint(15, 25) for i in range(len(x_data))]


line = Line(init_opts=opts.InitOpts(theme='light',
                                    width='1000px',
                                    height='600px'))
line.add_xaxis(x_data)
line.add_yaxis("Android",
               y_data_1,
               is_symbol_show=False,
               is_smooth=True,
               # 传入线风格参数
               linestyle_opts=line_style)
line.add_yaxis("IOS",
               y_data_2,
               is_symbol_show=False,
               is_smooth=True,
               # 传入线风格参数
               linestyle_opts=line_style)

# 添加标题
line.set_global_opts(title_opts=opts.TitleOpts(title="终端日活趋势"))

line.render_notebook()


## 阴影+径向渐变

* 在使用散点图/气泡图的时候，添加阴影和渐变效果能起到产生非常好的视觉效果；


In [4]:
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
from pyecharts.faker import Faker

# 渐变配色
# 通过js代码实现
# 大家记住这个代码就行，每次根据自己需求修改color中的值
color_js = """new echarts.graphic.RadialGradient(0.4, 0.3, 1, [{
                offset: 0,
                color: 'rgb(251, 118, 123)'
            }, {
                offset: 1,
                color: 'rgb(204, 46, 72)'
            }])"""

scatter = Scatter()
scatter.add_xaxis(Faker.choose())

scatter.add_yaxis(
    "红色",
    Faker.values(),
    symbol_size='30',
    # 渐变配色
    itemstyle_opts={
        'shadowBlur': 10,   # 光晕
        'shadowColor': 'rgba(0, 0, 0, 0.5)',  # 阴影颜色
        'shadowOffsetY': 5,  # 阴影偏移量——Y方向
        'shadowOffsetX': 5,  # 阴影偏移量——X方向
        'color': JsCode(color_js)  # 使用JsCode进行JS代码封装
    },
    label_opts=opts.LabelOpts(is_show=False),
)

scatter.render_notebook()


## 线性渐变

* 比较适合在柱形图中使用；

In [5]:
from pyecharts.charts import *
from pyecharts import options as opts
import random
from pyecharts.commons.utils import JsCode

# 渐变色JS代码
# 这个是径向渐变的代码
# 不会JS的记住就行，根据自己需求修改color对应的颜色值即可
color_js = """
            new echarts.graphic.LinearGradient(
                                0,
                                1,
                                0,
                                0,
                                [{offset: 0, color: '#00008B'},
                                 {offset: 1, color: '#DA70D6'}],
                                false)
           """
x_data = ["2020/7/{}".format(i + 1) for i in range(7)]

# 随机生成点数据
y_data = [random.randint(10, 20) for i in range(len(x_data))]


bar = Bar(
    init_opts=opts.InitOpts(
        theme='light',
        width='1000px',
        height='600px')
)
bar.add_xaxis(x_data)
bar.add_yaxis(
    '',
    y_data,
    itemstyle_opts={
        'shadowBlur': 10,   # 光晕
        'shadowColor': 'rgba(0, 0, 0, 0.5)',  # 阴影颜色
        'shadowOffsetY': 5,  # 阴影偏移量——Y方向
        'shadowOffsetX': 5,  # 阴影偏移量——X方向
        'color': JsCode(color_js)  # 通过JsCode封装JS代码
    },
)


bar.render_notebook()


## 直方图设置圆角

* 千篇一律的矩形直方图看着审美疲劳？其实我们可以让直方图变得更加圆润一些～

In [6]:
from pyecharts.charts import *
from pyecharts import options as opts
import random
from pyecharts.commons.utils import JsCode

color_js = """
            new echarts.graphic.LinearGradient(
                                0,
                                1,
                                0,
                                0,
                                [{offset: 0, color: '#00008B'},
                                 {offset: 1, color: '#DA70D6'}],
                                false)
           """
x_data = ["2020/7/{}".format(i + 1) for i in range(7)]

# 随机生成点数据
y_data = [random.randint(10, 20) for i in range(len(x_data))]


bar = Bar(
    init_opts=opts.InitOpts(
        theme='light',
        width='1000px',
        height='600px')
)
bar.add_xaxis(x_data)
bar.add_yaxis(
    '',
    y_data,
    itemstyle_opts={
        'shadowBlur': 10,   # 光晕
        'shadowColor': 'rgba(0, 0, 0, 0.5)',  # 阴影颜色
        'shadowOffsetY': 5,  # 阴影偏移量——Y方向
        'shadowOffsetX': 5,  # 阴影偏移量——X方向
        'barBorderRadius':[60, 60, 20, 20],  # 圆角效果 矩形的四个角对应的半径
        'color': JsCode(color_js)
    },
)


bar.render_notebook()


## 关闭坐标轴显示

还是继续在上面图的基础上更改，很多时候，当我们图表中已经有标签去标注数据的时候，这时候坐标轴看起来就没那么必要，这时我们便可以选择去关闭它；


In [7]:
from pyecharts.charts import *
from pyecharts import options as opts
import random
from pyecharts.commons.utils import JsCode

color_js = """
            new echarts.graphic.LinearGradient(
                                0,
                                1,
                                0,
                                0,
                                [{offset: 0, color: '#00008B'},
                                 {offset: 1, color: '#DA70D6'}],
                                false)
           """
x_data = ["2020/7/{}".format(i + 1) for i in range(7)]

# 随机生成点数据
y_data = [random.randint(10, 20) for i in range(len(x_data))]


bar = Bar(
    init_opts=opts.InitOpts(
        theme='light',
        width='1000px',
        height='600px')
)
bar.add_xaxis(x_data)
bar.add_yaxis(
    '',
    y_data,
    itemstyle_opts={
        'shadowBlur': 10,   # 光晕
        'shadowColor': 'rgba(0, 0, 0, 0.5)',  # 阴影颜色
        'shadowOffsetY': 5,  # 阴影偏移量——Y方向
        'shadowOffsetX': 5,  # 阴影偏移量——X方向
        'barBorderRadius':[60, 60, 20, 20],
        'color': JsCode(color_js)
    },
)

bar.set_global_opts(yaxis_opts=opts.AxisOpts(is_show=False))

bar.render_notebook()


## 坐标轴配置

* 其实除了单纯的关闭/开启显示，对于坐标轴能配置的东西是非常多的；

* 包括坐标轴线，刻度线，坐标轴标签，分割线等都可以进行样式配置；

* 下面以x轴为例，关闭坐标轴线，刻度线和分割线，并对坐标轴标签样式进行修改；

In [8]:
from pyecharts.charts import *
from pyecharts import options as opts
import random
from pyecharts.commons.utils import JsCode

color_js = """
            new echarts.graphic.LinearGradient(
                                0,
                                1,
                                0,
                                0,
                                [{offset: 0, color: '#00008B'},
                                 {offset: 1, color: '#DA70D6'}],
                                false)
           """
x_data = ["2020/7/{}".format(i + 1) for i in range(7)]

# 随机生成点数据
y_data = [random.randint(10, 20) for i in range(len(x_data))]

bar = Bar(
    init_opts=opts.InitOpts(
        theme='light',
        width='1000px',
        height='600px')
)
bar.add_xaxis(x_data)

bar.add_yaxis(
    '',
    y_data,
    itemstyle_opts={
        'shadowBlur': 10,   # 光晕
        'shadowColor': 'rgba(0, 0, 0, 0.5)',  # 阴影颜色
        'shadowOffsetY': 5,  # 阴影偏移量——Y方向
        'shadowOffsetX': 5,  # 阴影偏移量——X方向
        'barBorderRadius':[60, 60, 20, 20],
        'color': JsCode(color_js)
    },
)

bar.set_global_opts(
    yaxis_opts=opts.AxisOpts(is_show=False),   # 关闭Y轴显示
    xaxis_opts=opts.AxisOpts(
        boundary_gap=False,  # 两边不显示间隔
        axistick_opts=opts.AxisTickOpts(is_show=False),  # 刻度不显示
        splitline_opts=opts.SplitLineOpts(is_show=False),  # 分割线不显示
        axisline_opts=opts.AxisLineOpts(is_show=False),  # 轴线不显示
        axislabel_opts=opts.LabelOpts(  # 坐标轴标签配置
            font_size=20,  # 字体大小
            font_weight='bold'  # 加重
            )
    )
    )

bar.render_notebook()


## 标签样式配置

标签即图形上显示的问题部分，我们可以选择关闭其显示，也可以设置其文本风格，显示位置等等；

In [9]:
from pyecharts.charts import *
from pyecharts import options as opts
import random

color_js = """
            new echarts.graphic.LinearGradient(
                                0,
                                1,
                                0,
                                0,
                                [{offset: 0, color: '#00008B'},
                                 {offset: 1, color: '#DA70D6'}],
                                false)
           """
x_data = ["2020/7/{}".format(i + 1) for i in range(7)]

# 随机生成点数据
y_data = [random.randint(10, 20) for i in range(len(x_data))]

bar = Bar(
    init_opts=opts.InitOpts(
        theme='light',
        width='1000px',
        height='600px')
)
bar.add_xaxis(x_data)

bar.add_yaxis(
    '',
    y_data,
    itemstyle_opts={
        'shadowBlur': 10,   # 光晕
        'shadowColor': 'rgba(0, 0, 0, 0.5)',  # 阴影颜色
        'shadowOffsetY': 5,  # 阴影偏移量——Y方向
        'shadowOffsetX': 5,  # 阴影偏移量——X方向
        'barBorderRadius':[60, 60, 20, 20],
        'color': JsCode(color_js)
    },
    label_opts=opts.LabelOpts(  # 标签设置
        font_size=25,   # 字体大小
        font_weight='bold',  # 加粗
        position='insideTop'  # 标签位置
        ),
)

bar.set_global_opts(
    yaxis_opts=opts.AxisOpts(is_show=False),   # 关闭Y轴显示
    xaxis_opts=opts.AxisOpts(
        boundary_gap=False,  # 两边不显示间隔
        axistick_opts=opts.AxisTickOpts(is_show=False),  # 刻度不显示
        splitline_opts=opts.SplitLineOpts(is_show=False),  # 分割线不显示
        axisline_opts=opts.AxisLineOpts(is_show=False),  # 轴线不显示
        axislabel_opts=opts.LabelOpts(  # 坐标轴标签配置
            font_size=20,  # 字体大小
            font_weight='bold'  # 加重
            )
    )
    )

bar.render_notebook()


## 背景颜色

* 和图形颜色配置一致，背景颜色同样可以支持配置渐变色；

* 当然除此之外，在这一步中，我还偷偷改了坐标轴标签的颜色+加了白色边框；

In [10]:
from pyecharts.charts import *
from pyecharts import options as opts
import random

color_js = """
            new echarts.graphic.LinearGradient(
                                0,
                                1,
                                0,
                                0,
                                [{offset: 0, color: '#00008B'},
                                 {offset: 1, color: '#DA70D6'}],
                                false)
           """
x_data = ["2020/7/{}".format(i + 1) for i in range(7)]

# 随机生成点数据
y_data = [random.randint(10, 20) for i in range(len(x_data))]

bar = Bar(
    init_opts=opts.InitOpts(
        bg_color= JsCode(color_js),
        width='1000px',
        height='600px')
)
bar.add_xaxis(x_data)

bar.add_yaxis(
    '',
    y_data,
    itemstyle_opts={
        'shadowBlur': 10,   # 光晕
        'shadowColor': 'rgba(0, 0, 0, 0.5)',  # 阴影颜色
        'shadowOffsetY': 5,  # 阴影偏移量——Y方向
        'shadowOffsetX': 5,  # 阴影偏移量——X方向
        'barBorderRadius':[60, 60, 20, 20],
        'borderColor': 'white',  # 边框颜色
        'borderWidth': 1,
        'color': JsCode(color_js)
    },
    label_opts=opts.LabelOpts(  # 标签设置
        font_size=25,   # 字体大小
        font_weight='bold',  # 加粗
        position='insideTop'  # 标签位置
        ),
)

bar.set_global_opts(
    yaxis_opts=opts.AxisOpts(is_show=False),   # 关闭Y轴显示
    xaxis_opts=opts.AxisOpts(
        boundary_gap=False,  # 两边不显示间隔
        axistick_opts=opts.AxisTickOpts(is_show=False),  # 刻度不显示
        splitline_opts=opts.SplitLineOpts(is_show=False),  # 分割线不显示
        axisline_opts=opts.AxisLineOpts(is_show=False),  # 轴线不显示
        axislabel_opts=opts.LabelOpts(  # 坐标轴标签配置
            font_size=20,  # 字体大小
            font_weight='bold',  # 加重
            color='white'  # 字体颜色
            )
    )
    )

bar.render_notebook()


## 添加文本框

有时候需要对图表添加文字说明等，就需要在图表上添加一个文本框；

In [11]:
from pyecharts.charts import *
from pyecharts import options as opts
import random

color_js = """
            new echarts.graphic.LinearGradient(
                                0,
                                1,
                                0,
                                0,
                                [{offset: 0, color: '#00008B'},
                                 {offset: 1, color: '#DA70D6'}],
                                false)
           """
x_data = ["2020/7/{}".format(i + 1) for i in range(7)]

# 随机生成点数据
y_data = [random.randint(10, 20) for i in range(len(x_data))]

bar = Bar(
    init_opts=opts.InitOpts(
        bg_color=JsCode(color_js),
        width='1000px',
        height='600px')
)
bar.add_xaxis(x_data)

bar.add_yaxis(
    '',
    y_data,
    itemstyle_opts={
        'shadowBlur': 10,   # 光晕
        'shadowColor': 'rgba(0, 0, 0, 0.5)',  # 阴影颜色
        'shadowOffsetY': 5,  # 阴影偏移量——Y方向
        'shadowOffsetX': 5,  # 阴影偏移量——X方向
        'barBorderRadius':[60, 60, 20, 20],
        'borderColor': 'white',  # 边框颜色
        'borderWidth': 1,
        'color': JsCode(color_js)
    },
    label_opts=opts.LabelOpts(  # 标签设置
        font_size=25,   # 字体大小
        font_weight='bold',  # 加粗
        position='insideTop'  # 标签位置
        ),
)

bar.set_global_opts(
    yaxis_opts=opts.AxisOpts(is_show=False),   # 关闭Y轴显示
    xaxis_opts=opts.AxisOpts(
        boundary_gap=False,  # 两边不显示间隔
        axistick_opts=opts.AxisTickOpts(is_show=False),  # 刻度不显示
        splitline_opts=opts.SplitLineOpts(is_show=False),  # 分割线不显示
        axisline_opts=opts.AxisLineOpts(is_show=False),  # 轴线不显示
        axislabel_opts=opts.LabelOpts(  # 坐标轴标签配置
            font_size=20,  # 字体大小
            font_weight='bold',  # 加重
            color='white'  # 字体颜色
            )
    ),
    graphic_opts=[opts.GraphicGroup(
        graphic_item=opts.GraphicItem(
            # 将组件进行旋转
            rotation=JsCode("Math.PI / 4"),
            bounding="raw",
            right=110,
            bottom=110,
            z=2),
        # 文字显示区域配置
        children=[
            # 文字框
            opts.GraphicRect(
                graphic_item=opts.GraphicItem(
                    left="center", top="center", z=100
                ),
                graphic_shape_opts=opts.GraphicShapeOpts(
                    width=400, height=50
                ),
                graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(
                    # 颜色配置，这里设置为黑色，透明度为0.5
                    fill="rgba(0,0,0,0.3)"
                ),
            ),
            # 文本设置
            opts.GraphicText(
                graphic_item=opts.GraphicItem(
                    left="center", top="center", z=100
                ),
                graphic_textstyle_opts=opts.GraphicTextStyleOpts(
                    # 要显示的文本
                    text='AwesomeTang',
                    font="bold 26px Microsoft YaHei",
                    graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(
                        fill="#fff"
                    ),
                ),
            ),
        ],
    )
    ],
)

bar.render_notebook()


## 视觉组件（多个指标）

* 通过视觉组件来反映数值大小，这一块大部分小伙伴都熟悉；

* 不过如果想同时映射到多个指标该如何操作呢，比如同时使用透明度，颜色，大小来反映数据；

In [12]:
from pyecharts.charts import *
from pyecharts import options as opts
import random

x_data = [random.randint(0, 100) for _ in range(30)]
# 注意ydata中包含多个维度的数据
y_data = [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100), random.randint(0, 100))
          for _ in range(30)]

# 新建散点图
scatter = Scatter(init_opts=opts.InitOpts(theme='light',
                                          width='1000px',
                                          height='600px'))
# 添加x轴数据
scatter.add_xaxis(x_data)
# 添加y轴数据
scatter.add_yaxis(
    '', 
    y_data
    )
# dimension指定数据维度
scatter.set_global_opts(visualmap_opts=[opts.VisualMapOpts(is_show=True, type_='size', dimension=2, pos_top='5%', range_text=['大小', '']),  # 通过图形大小映射
                                        opts.VisualMapOpts(is_show=True, type_='color', dimension=3, pos_top='35%', range_text=['颜色', '']),   # 通过颜色映射
                                        opts.VisualMapOpts(is_show=True, type_='opacity', dimension=4, range_opacity=[0.2, 1], pos_top='65%', range_text=['透明度', ''])  # 通过透明度映射
                                        ],
    xaxis_opts=opts.AxisOpts(type_="value"))
scatter.render_notebook()


## Task 今日份小作业

* **期望效果**


![Image Name](https://cdn.kesci.com/upload/image/qywiqndeif.png?imageView2/0/w/960/h/960)


成品不需要一模一样，但至少得保证以下功能效果的实现：

* 堆叠效果；

* 渐变配色；

* 面积图的实现；

---

***附：需要数据***

In [13]:
x_data = ['周一', '周二', '周三', '周四', '周五', '周六', '周日']
y1 = [140, 232, 101, 264, 90, 340, 250]
y2 = [120, 282, 111, 234, 220, 340, 310]
y3 = [320, 132, 201, 334, 190, 130, 220]
y4 = [220, 402, 231, 134, 190, 230, 120]
y5 = [220, 302, 181, 234, 210, 290, 150]
